In [2]:
%serialconnect


Connecting to Serial /dev/ttyUSB0 baud=115200

 ** Serial connected **

Serial<id=0x7fc5dea57b70, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
normal repl mode not detected b''
not command mode
.ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0010,len:4
load:0x3fff0014,len:4268
load:0x40078000,len:0
load:0x40078000,len:10648
entry 0x4007a56c
OSError: [Errno 2] ENOENT
MicroPython v1.9.2-273-g56f18602 on 2017-09-25; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' 

In [85]:
%%writefile pca9685servo.py

import machine, time, math, ustruct

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
assert 0x40 in i2c.scan(), "No PCA9685 connected to the I2C bus"

def _write(address, value):
    i2c.writeto_mem(0x40, address, bytearray([value]))

def _read(address):
    return i2c.readfrom_mem(0x40, address, 1)[0]

i2c.writeto_mem(0x40, 0x40, b'\x00')   # Mode1

# set frequency
freq = 50
prescale = int(25000000.0 / 4096.0 / freq + 0.5)
old_mode = _read(0x00) # Mode 1
_write(0x00, (old_mode & 0x7F) | 0x10) # Mode 1, sleep
_write(0xfe, prescale) # Prescale
_write(0x00, old_mode) # Mode 1
time.sleep_us(5)
_write(0x00, old_mode | 0xa1) # Mode 1, autoincrement on


def setang(i, g):
    pwmduty = int((0.5 + g/180*2.0)/(1000/freq)*4096)    
    data = ustruct.pack('<HH', 0, pwmduty)
    i2c.writeto_mem(0x40, 0x06 + 4 * i,  data)

def getang(i):
    data = i2c.readfrom_mem(0x40, 0x06 + 4 * i, 4)
    pwmduty1 = ustruct.unpack('<HH', data)
    pwmduty = pwmduty1[1]
    return (pwmduty/4096*(1000/freq)-0.5)/2*180

def setangspeed(vpos, degreespersecond = 500):
    if degreespersecond == 0:
        for i in range(len(vpos)):
            setang(i, vpos[i])
        return
    cpos = [ getang(i)  for i in range(len(vpos)) ]
    n = max(abs(c-v)  for c, v in zip(cpos, vpos))
    for d in range(1, n+1):
        for i in range(len(vpos)):
            setang(i, cpos[i] + (d/n)*(vpos[i] - cpos[i]))
            time.sleep(1/degreespersecond)

            

Writing pca9685servo.py



In [82]:
%sendtofile --source pca9685servo.py

In [88]:
%disconnect
